In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.utils.data as Data  
import matplotlib.pyplot as plt

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


# Load the input tensor we saved in earlier practice

In [7]:
# train_x = torch.load('train_x.tensor')
train_x = torch.load('https://vriemyzuedutw-my.sharepoint.com/:u:/g/personal/tsun2611_vr_iem_yzu_edu_tw/EXyy9G5X1QVDguZO1dtSVj4BoZ_NlYg-aqLHTi-Ntc05CQ?e=xTwoGi')

FileNotFoundError: [Errno 2] No such file or directory: 'https://vriemyzuedutw-my.sharepoint.com/:u:/g/personal/tsun2611_vr_iem_yzu_edu_tw/EXyy9G5X1QVDguZO1dtSVj4BoZ_NlYg-aqLHTi-Ntc05CQ?e=xTwoGi'

In [4]:
train_y = torch.load('train_y.tensor')

In [ ]:
train_x.shape

In [ ]:
train_y.shape

# Reshape train_y from 25000x1 to 25000

In [ ]:
train_y = train_y.view(25000)

In [ ]:
train_y.shape

# Build training tensor data set and data loader

In [ ]:
train_x = train_x.type(torch.FloatTensor)

In [ ]:
train_y = train_y.type(torch.LongTensor)

In [ ]:
torch_dataset = Data.TensorDataset(data_tensor=train_x, target_tensor=train_y)

In [ ]:
loader = Data.DataLoader(
    dataset=torch_dataset,     # torch TensorDataset format
    batch_size=5000,           # mini batch size
    shuffle=True,               # random shuffle for training
    num_workers=2,              # subprocesses for loading data
)

# Define NN

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=240,
            hidden_size=64,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(64, 2)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out

In [ ]:
rnn = RNN()

In [ ]:
rnn

# Run one training epoc step by step to understand details

In [ ]:
for step, (x, y) in enumerate(loader):
    break

In [ ]:
print(step, x.shape, y.shape)

In [ ]:
b_x = Variable(x.view(-1, 80, 80*3))            # reshape x to (batch, time_step, input_size)
b_y = Variable(y)       
print(b_x.shape, b_y.shape)

In [ ]:
output = rnn(b_x) 

In [ ]:
output.shape

In [ ]:
loss = loss_func(output, b_y) 

In [ ]:
loss

In [ ]:
loss.data.numpy()[0]

# Training loop

In [ ]:
losslst = []
for epoch in range(1):  #run only one epoc
    for step, (x, y) in enumerate(loader):        # gives batch data
        b_x = Variable(x.view(-1, 80, 80*3))            # reshape x to (batch, time_step, input_size)
        b_y = Variable(y)                               # batch y

        output = rnn(b_x)                               # rnn output
        loss = loss_func(output, b_y)                   # cross entropy loss
        
        print('Epoch: ', epoch, '| step: ', step, '| train loss: ', loss.data[0])
        losslst.append(loss.data[0])
        
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

# Visualize loss plot

In [ ]:
plt.figure(figsize=(18,9)) # set the figure size to 18 x 9 inches
plt.plot(losslst, color = 'red')
plt.show()